In [1]:
import pandas as pd
import subprocess
import os
import shutil

In [2]:
def get_video_duration(file_path):
    command = ['ffmpeg', '-i', file_path]
    result = subprocess.run(command, text=True, capture_output=True)
    output_lines = result.stderr.split('\n')
    duration_line = [line for line in output_lines if 'Duration' in line][0]
    duration = duration_line.strip().split(",")[0].split(" ")[1]
    return duration

In [3]:
def create_folder(folder_name):
    try:
        # Create a new folder in the current working directory
        os.mkdir(folder_name)
        print(f"Folder '{folder_name}' created successfully.")
    except FileExistsError:
        print(f"Folder '{folder_name}' already exists.")

In [4]:
def get_the_table_data():
    while True:
        table_path = input("What is the table path + name? (.../filename.csv):")
        try:
            video_schnitt_df = pd.read_csv(table_path)
            print("Table loaded succesfully")
            return video_schnitt_df
        except Exception as error:
            print("An error occured:", error)
            continue
        break

In [5]:
def clean_the_data(df):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(" ", "_")
    df.columns = df.columns.str.strip()
    df = df.astype(str)
    print("Table Columns cleaned")
    return df

In [6]:
def cut_head_tail(original_video,output_file,cut_head="00:00:00",cut_tail="00:50:00"):

    # Construct the command
    command = [
        'ffmpeg',
        '-i', original_video,
        '-ss', cut_head,
        '-to', cut_tail,
        '-c:v','libx264',
         '-c:a', 'copy',
        output_file
    ]
    
     # Run the command
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

In [7]:
def concatenate_videos(input_file, output_file):
    
    command = [
        'ffmpeg',
        '-f', 'concat', 
        '-i', input_file,
         '-c:v', 'copy',
         '-c:a', 'copy',
        output_file
    ]

    # Run the command
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print("An error occured:", e)

# Get the file and clean it

In [8]:
video_schnitt_df = get_the_table_data()
video_schnitt_df = clean_the_data(video_schnitt_df)

Table loaded succesfully
Table Columns cleaned


# remove more slices in a video

In [11]:
try:
    # videos need to be in same directory with python script
    for idx, video_name in video_schnitt_df["dateiname"].items():
        
        columns_to_cut = ["vorne_abschneiden_bis","rausschneiden_ab","rausschneiden_bis","cut2_ab","cut2_bis","cut3_ab","cut3_bis","cut4_ab","cut4_bis","cut5_ab","cut5_bis"]

        parts_list = []
        video1 = 0
        video2 = 0

        try:
            for i,col in enumerate(columns_to_cut):
                cut_head = video_schnitt_df[col][idx]
                cut_tail = video_schnitt_df[columns_to_cut[i+1]][idx]

                # logic to handle the nans
                if cut_head == "nan" and cut_tail == "nan":
                    break
                elif cut_head == "nan":
                    cut_head = "00:00:00"
                elif cut_tail == "nan":
                    cut_tail = get_video_duration(video_name)

                output_file = f"{video_name.split('.')[0]}_part{i}.mp4"
                print("video_name:",video_name,"cut times:",cut_head,cut_tail)
                #cut_head_tail(video_name, output_file, cut_head, cut_tail)
                
                if i not in [1, 3, 5, 7, 9]:
                    parts_list.append(output_file)

                if len(parts_list) == 2:
                    video1 = parts_list[0]
                    video2 = parts_list[1]
                    
                    textfile_content = f"file '{video1}'\nfile '{video2}'"

                    #create the file with content
                    with open('dummy.txt', 'w') as textfile:
                        textfile.write(textfile_content)

                    #define parameters for concat function
                    textfile_name = "dummy.txt"

                    output_file = f"{video_name.split('.')[0]}_concatted{i}.mp4"
                    
                    parts_list.clear()
                    parts_list.append(output_file)
                    print(textfile_content, "output:", output_file)
                    #concatenate_videos(textfile_name,output_file)                

        except:
            print("skip to next video")

except Exception as error:
    print("An error occured:", error)   

video_name: AD_Streuung.mp4 cut times: 00:00:08.000 00:02:05.000
video_name: AD_Streuung.mp4 cut times: 00:02:05.000 00:03:05.000
video_name: AD_Streuung.mp4 cut times: 00:03:05.000 00:03:30.000
file 'AD_Streuung_part0.mp4'
file 'AD_Streuung_part2.mp4' output: AD_Streuung_concatted2.mp4
skip to next video
video_name: Chi_Quadrat_Klausuraufgabe.mp4 cut times: 00:01:40.000 00:03:06.000
video_name: Chi_Quadrat_Klausuraufgabe.mp4 cut times: 00:03:06.000 00:04:19.000
video_name: Chi_Quadrat_Klausuraufgabe.mp4 cut times: 00:04:19.000 00:05:47.000
file 'Chi_Quadrat_Klausuraufgabe_part0.mp4'
file 'Chi_Quadrat_Klausuraufgabe_part2.mp4' output: Chi_Quadrat_Klausuraufgabe_concatted2.mp4
skip to next video
video_name: Chi_Quadrat.mp4 cut times: 00:00:04.200 00:08:15.000
video_name: Chi_Quadrat.mp4 cut times: 00:08:15.000 00:09:22.000
video_name: Chi_Quadrat.mp4 cut times: 00:09:22.000 00:10:59.000
file 'Chi_Quadrat_part0.mp4'
file 'Chi_Quadrat_part2.mp4' output: Chi_Quadrat_concatted2.mp4
skip to 

In [ ]:
path_script_output = "script_output"
create_folder(path_script_output)

path_folder_parts = "script_output/parts"
create_folder(path_folder_parts)

In [ ]:
original_video_names = list()

for video_name in video_schnitt_df["dateiname"]:
    if video_name != "nan":
        original_video_names.append(video_name)

In [ ]:
for filename in os.listdir():
    if filename.endswith(".mp4") and filename not in original_video_names:
        shutil.move(filename,path_script_output)

In [ ]:
for filename in os.listdir(path_script_output):
    if filename.find("concatted"):
        src = f"script_output/{filename}"
        shutil.move(src,path_folder_parts)

In [ ]:
for filename in os.listdir(path_script_output):
    if filename in ["Interquartilsabstand_part0.mp4","Korrelation_Werttabelle_part_0.mp4","Mittlere_absoulte_Abweichung_vom_Median_part0.mp4"]:
        src = f"script_output/{filename}"
        shutil.move(src,path_folder_parts)